<a href="https://colab.research.google.com/github/e-learn-12/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_4%EA%B0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 오픈소스 기반 데이터 분석 4강 - 데이터 수집


## 4-1 CSV 파일 읽기

In [ ]:
import pandas as pd

## data.csv 파일 읽기
df = pd.read_csv('data.csv', encoding='utf-8', sep=',', header=0, index_col=None, skiprows=None, nrows=None)
print(df)

           날짜    체중  골격근량  체지방량
0  2025.02.06  64.7  30.0  11.1
1  2025.02.04  64.0  29.3  11.6


## 4-2 JSON 파일 읽기



In [ ]:
import json
import pandas as pd

## data.json 파일 출력
with open('data.json', mode='r', encoding='utf-8') as f:
    data = json.load(f)
    print(f'json.load 사용--------------\n{data}\n\n')
## data.json 파일 DataFrame 읽기
df = pd.read_json('data.json', orient='records', encoding='utf-8', lines=False)
print(f'pandas.read_json 사용--------------\n{df}')

json.load 사용--------------
{'매출데이터': [{'월': '2025-01', '매출액': 1000000, '비용': 700000, '이익': 300000}, {'월': '2025-02', '매출액': 1200000, '비용': 800000, '이익': 400000}, {'월': '2025-03', '매출액': 1500000, '비용': 900000, '이익': 600000}]}


pandas.read_json 사용--------------
                                               매출데이터
0  {'월': '2025-01', '매출액': 1000000, '비용': 700000,...
1  {'월': '2025-02', '매출액': 1200000, '비용': 800000,...
2  {'월': '2025-03', '매출액': 1500000, '비용': 900000,...


## 4-3 텍스트 파일 읽기 및 데이터 추출

In [ ]:
import re

## 파일(callcenter20250301.log) 오픈 및 읽기
with open('callcenter20250301.log', 'r', encoding='utf-8') as f:
    content = f.read()
## 주민등록번호 패턴 생성
pattern = re.compile(r'(\d{6})-(\d{7})')

## 주민등록번호 마스킹
masked_content = pattern.sub(r'\1-*******', content)

## 마스킹된 파일(callcenter20250301_masked.log) 오픈 및 쓰기
with open('callcenter20250301_masked.log', mode='w') as f:
    f.write(masked_content)

print("주민등록번호 마스킹 완료. 'callcenter20250301_masked.log' 파일로 저장되었습니다.")

## 4-4 Open-Meteo의 무료 날씨 API를 통한 특정 지역 온도 조회

In [ ]:
import requests
import json

url = "https://api.open-meteo.com/v1/forecast?=&=&current=temperature_2m"
params = {
    "latitude": "37.58638333",
    "longitude": "127.0203333",
    "current": "temperature_2m"
}

try:
    ## URL 및 파라미터 전송
    response = requests.get(url, params=params)
    response.raise_for_status() # 200 OK가 아니면 예외발생

    ## JSON 데이터 읽기
    data = response.json()

    print("API 응답:", data)
    print("서울시 종로구의 현재 온도는 : {0}{1} 입니다.".format(data['current']['temperature_2m'], data['current_units']['temperature_2m']))

except requests.exceptions.RequestException as e:
    print(f"API 호출 실패: {e}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 실패: {e}")

API 응답: {'latitude': 37.6, 'longitude': 127.0, 'generationtime_ms': 0.02849102020263672, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 29.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C'}, 'current': {'time': '2025-10-19T07:00', 'interval': 900, 'temperature_2m': 17.8}}
서울시 종로구의 현재 온도는 : 17.8°C 입니다.


## 4-5 Selenium과 lxml을 이용한 웹 스크래핑

In [ ]:
!curl -o google-chrome-stable_current_amd64.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install ./google-chrome-stable_current_amd64.deb -y
!pip install selenium webdriver_manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from lxml import html
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')               # 브라우저 창 없이 실행
chrome_options.add_argument('--no-sandbox')             # 보안모드 비활성화 (Colab 필수)
chrome_options.add_argument('--disable-dev-shm-usage')  # 메모리 부족 방지 (Colab 필수)
chrome_options.add_argument('--window-size=1920x1080')  # 창 크기 설정(가상)
chrome_options.add_argument('--disable-gpu')            # GPU 가속 비활성화 (일부 환경 안정성)
chrome_options.binary_location = "/usr/bin/google-chrome-stable"  # Colab용 크롬 경로 지정

## 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)

## 사이트 접속
url = 'https://professor.knou.ac.kr/jaehwachung//index.do'
driver.get(url)

## 사이트 접속 대기
time.sleep(2)

## 페이지 제목 출력
page_source = driver.page_source
tree = html.fromstring(page_source)

title_text = tree.xpath('//title/text()')
print(title_text)

## 드라이버 종료
driver.quit()

<Element html at 0x7bfc5e4e9fe0>
['\n\t\tAlert \n\t\t\n\t']



# 실습 시나리오

## 공공데이터 포털 가입 및 데이터 신청

- [https://www.data.go.kr](https://www.data.go.kr)
- 한국환경공단 에어코리아 대기오염정보 데이터 신청

In [ ]:
import requests

## 데이터 수집 url 및 api key 설정


params = {
    'serviceKey': api_key,
    'returnType': 'json',
    'numOfRows': '100',
    'pageNo': '1',
    'sidoName': '서울',
    'ver': '1.0'
}

## 데이터 수집

## 호출 성공/실패 출력



In [ ]:
import requests
import json
import time

api_key = '발급받은키'
years = [str(year) for year in range(2014, 2025)]
months = [str(m+1).zfill(2) for m in range(12)]
data_result = []

# 년도, 월, 회원타입, 현년 전기사용량, 현년 가스사용량, 현년 수도사용량, 현년 지역난방 사용량
columns = ['YEAR', 'MON', 'MM_TYPE', 'EUS', 'GUS', 'WUS', 'HUS']

try:
    ## URL 및 파라미터 전송

    for year in years:
        for month in months:
            url = f'http://openapi.seoul.go.kr:8088/{api_key}/json/energyUseDataSummaryInfo/1/5/{year}/{month}'
            response = requests.get(url)
            response.raise_for_status() # 200 OK가 아니면 예외발생

            ## JSON 데이터 읽기
            data = response.json()
            print(data)

            # 개인 유형의 데이터만 data_result 에 저장하기
            rows = data.get('energyUseDataSummaryInfo', {}).get('row', [])
            get_type_personal = lambda r : r['MM_TYPE'] == '개인'
            filtered_list = list(filter(get_type_personal, rows))

            # 년도, 월, 회원타입, 현년 전기, 가스, 수도, 지역난방 컬럼만 저장하기
            compact_list = [{col:row[col] for col in columns} for row in filtered_list]
            data_result.extend(compact_list)
            time.sleep(0.4)

    # '개인'유형의 데이터 data_result를 json으로 저장하기
    with open('data_result.json', mode='w') as json_file:
        json.dump(data_result, json_file, indent=4, ensure_ascii=False)

except requests.exceptions.RequestException as e:
    print(f"API 호출 실패: {e}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 실패: {e}")

In [ ]:
import pandas as pd

month_to_season = {
    12: '겨울', 1: '겨울', 2: '겨울',
    3: '봄', 4: '봄', 5: '봄',
    6: '여름', 7: '여름', 8: '여름',
    9: '가을', 10: '가을', 11: '가을'
}

## data_result.json 파일을 DataFrame으로 읽기
df = pd.read_json('data_result.json', orient='records', encoding='utf-8', lines=False)
season_data = df["MON"].astype(int).map(month_to_season);
df.insert(df.columns.get_loc('YEAR') + 1, 'SEASON', season_data)

print(df)

    YEAR SEASON  MON MM_TYPE        EUS         GUS           WUS  \
0   2014     겨울    1      개인  144900337  38335224.0  1.061396e+07   
1   2014      봄    3      개인  135419507  32209021.0  1.106945e+07   
2   2014     여름    7      개인  168681507   7318863.0  1.313594e+07   
3   2014     여름    8      개인  187725750   6596679.0  1.403165e+07   
4   2014     가을    9      개인  175409826   6969566.0  1.383068e+07   
..   ...    ...  ...     ...        ...         ...           ...   
85  2024     여름    8      개인  463971328   7072251.0  1.587354e+07   
86  2024     가을    9      개인  409872908   5729131.0  1.514332e+07   
87  2024     가을   10      개인  282268119   8233421.0  1.529767e+07   
88  2024     가을   11      개인  257927956  17418221.0  1.467867e+07   
89  2024     겨울   12      개인  134828126  42103424.0  7.326854e+06   

             HUS  
0   1.763254e+07  
1   1.047034e+07  
2   2.178534e+05  
3   3.205539e+05  
4   1.041888e+06  
..           ...  
85  1.300823e+05  
86  3.420507e+06  


In [ ]:
import pandas as pd

## data_result.json 파일을 DataFrame으로 읽기
df = pd.read_json('data_result.json', orient='records', encoding='utf-8', lines=False)
print(df)

    YEAR  MON MM_TYPE        EUS         GUS           WUS           HUS
0   2014    1      개인  144900337  38335224.0  1.061396e+07  1.763254e+07
1   2014    3      개인  135419507  32209021.0  1.106945e+07  1.047034e+07
2   2014    7      개인  168681507   7318863.0  1.313594e+07  2.178534e+05
3   2014    8      개인  187725750   6596679.0  1.403165e+07  3.205539e+05
4   2014    9      개인  175409826   6969566.0  1.383068e+07  1.041888e+06
..   ...  ...     ...        ...         ...           ...           ...
85  2024    8      개인  463971328   7072251.0  1.587354e+07  1.300823e+05
86  2024    9      개인  409872908   5729131.0  1.514332e+07  3.420507e+06
87  2024   10      개인  282268119   8233421.0  1.529767e+07  2.695491e+06
88  2024   11      개인  257927956  17418221.0  1.467867e+07  8.836296e+06
89  2024   12      개인  134828126  42103424.0  7.326854e+06  1.513199e+06

[90 rows x 7 columns]


In [ ]:
import json

# json_str = '{"energyUseDataSummaryInfo": {"list_total_count": 7, "RESULT": {"CODE": "INFO-000", "MESSAGE": "정상 처리되었습니다"}, "row": [{"YEAR": "2020", "MON": "12", "MM_TYPE": "공공기관", "CNT": "2319", "EUS": "134415822", "EUS1": "140830293", "EUS2": "143785616", "ECO2_1": "-7892132.5", "ECO2_2": "-3346264.18", "GUS": "3625280", "GUS1": "3938720", "GUS2": "4530840", "GCO2_1": "-609500", "GCO2_2": "-1365280", "WUS": "682469.6", "WUS1": "966115.2", "WUS2": "923345", "WCO2_1": "-262260.5", "WCO2_2": "-87070.486", "HUS": "0", "HUS1": "0", "HUS2": "0", "HCO2_1": "0", "HCO2_2": "0", "REG_DATE": "2021-03-11 18:31:32.0"}, {"YEAR": "2020", "MON": "12", "MM_TYPE": "기업", "CNT": "18259", "EUS": "567429666", "EUS1": "586794781", "EUS2": "593462122", "ECO2_1": "-22698785.5", "ECO2_2": "-9624285.052", "GUS": "24795758", "GUS1": "21567744", "GUS2": "26868220", "GCO2_1": "577776", "GCO2_2": "1294218.24", "WUS": "4056394.2", "WUS1": "4768906.8", "WUS2": "4628830", "WCO2_1": "-642474.2", "WCO2_2": "-213301.4344", "HUS": "0", "HUS1": "0", "HUS2": "0", "HCO2_1": "0", "HCO2_2": "0", "REG_DATE": "2021-03-11 18:31:32.0"}, {"YEAR": "2020", "MON": "12", "MM_TYPE": "소상공인", "CNT": "42743", "EUS": "82544954", "EUS1": "88007794", "EUS2": "87941049", "ECO2_1": "-5429467.5", "ECO2_2": "-2302094.22", "GUS": "2749449", "GUS1": "3195669", "GUS2": "3116983", "GCO2_1": "-406877", "GCO2_2": "-911404.48", "WUS": "1522971.8", "WUS1": "1710259.8", "WUS2": "1644036.2", "WCO2_1": "-154176.2", "WCO2_2": "-51186.4984", "HUS": "2679.61", "HUS1": "2096.78", "HUS2": "2244.49", "HCO2_1": "508.975", "HCO2_2": "-73.51022", "REG_DATE": "2021-03-11 18:31:32.0"}, {"YEAR": "2020", "MON": "12", "MM_TYPE": "학교", "CNT": "6457", "EUS": "148641247", "EUS1": "163043674", "EUS2": "158464391", "ECO2_1": "-12112785.5", "ECO2_2": "-5135821.052", "GUS": "6130182", "GUS1": "6712287", "GUS2": "6946755", "GCO2_1": "-699339", "GCO2_2": "-1566519.36", "WUS": "1849950.6", "WUS1": "2254096.6", "WUS2": "2179161.8", "WCO2_1": "-366678.6", "WCO2_2": "-121737.2952", "HUS": "26803.56", "HUS1": "21972.121", "HUS2": "24090.481", "HCO2_1": "3772.259", "HCO2_2": "6446.968818", "REG_DATE": "2021-03-11 18:31:32.0"}, {"YEAR": "2020", "MON": "12", "MM_TYPE": "개인", "CNT": "1110223", "EUS": "265370372", "EUS1": "250887378", "EUS2": "252983409", "ECO2_1": "13434978.5", "ECO2_2": "5696430.884", "GUS": "51095720", "GUS1": "50449214", "GUS2": "50298542", "GCO2_1": "721842", "GCO2_2": "1616926.08", "WUS": "15036546.3", "WUS1": "14479510.234", "WUS2": "14356727.09", "WCO2_1": "618427.638", "WCO2_2": "205317.975816", "HUS": "26512966.052", "HUS1": "23787438.214", "HUS2": "24469734.461", "HCO2_1": "2384379.7145", "HCO2_2": "4399125.3465152", "REG_DATE": "2021-03-11 18:31:32.0"}]}}'
json_str = '{"energyUseDataSummaryInfo": {"list_total_count": 0, "RESULT": {"CODE": "INFO-000", "MESSAGE": "정상 처리되었습니다"}, "row": []}}'
data = json.loads(json_str)
rows = data.get('energyUseDataSummaryInfo', {}).get('row', [])
print(data)
get_type_personal = lambda r : r['MM_TYPE'] == '개인'
p_data = list(filter(get_type_personal, rows))
print(p_data)

{'energyUseDataSummaryInfo': {'list_total_count': 0, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': []}}
[]


In [ ]:
import json

json_str = '{"energyUseDataSummaryInfo": {"list_total_count": 7, "RESULT": {"CODE": "INFO-000", "MESSAGE": "정상 처리되었습니다"}, "row": [{"YEAR": "2020", "MON": "12", "MM_TYPE": "공공기관", "CNT": "2319", "EUS": "134415822", "EUS1": "140830293", "EUS2": "143785616", "ECO2_1": "-7892132.5", "ECO2_2": "-3346264.18", "GUS": "3625280", "GUS1": "3938720", "GUS2": "4530840", "GCO2_1": "-609500", "GCO2_2": "-1365280", "WUS": "682469.6", "WUS1": "966115.2", "WUS2": "923345", "WCO2_1": "-262260.5", "WCO2_2": "-87070.486", "HUS": "0", "HUS1": "0", "HUS2": "0", "HCO2_1": "0", "HCO2_2": "0", "REG_DATE": "2021-03-11 18:31:32.0"}, {"YEAR": "2020", "MON": "12", "MM_TYPE": "기업", "CNT": "18259", "EUS": "567429666", "EUS1": "586794781", "EUS2": "593462122", "ECO2_1": "-22698785.5", "ECO2_2": "-9624285.052", "GUS": "24795758", "GUS1": "21567744", "GUS2": "26868220", "GCO2_1": "577776", "GCO2_2": "1294218.24", "WUS": "4056394.2", "WUS1": "4768906.8", "WUS2": "4628830", "WCO2_1": "-642474.2", "WCO2_2": "-213301.4344", "HUS": "0", "HUS1": "0", "HUS2": "0", "HCO2_1": "0", "HCO2_2": "0", "REG_DATE": "2021-03-11 18:31:32.0"}, {"YEAR": "2020", "MON": "12", "MM_TYPE": "소상공인", "CNT": "42743", "EUS": "82544954", "EUS1": "88007794", "EUS2": "87941049", "ECO2_1": "-5429467.5", "ECO2_2": "-2302094.22", "GUS": "2749449", "GUS1": "3195669", "GUS2": "3116983", "GCO2_1": "-406877", "GCO2_2": "-911404.48", "WUS": "1522971.8", "WUS1": "1710259.8", "WUS2": "1644036.2", "WCO2_1": "-154176.2", "WCO2_2": "-51186.4984", "HUS": "2679.61", "HUS1": "2096.78", "HUS2": "2244.49", "HCO2_1": "508.975", "HCO2_2": "-73.51022", "REG_DATE": "2021-03-11 18:31:32.0"}, {"YEAR": "2020", "MON": "12", "MM_TYPE": "학교", "CNT": "6457", "EUS": "148641247", "EUS1": "163043674", "EUS2": "158464391", "ECO2_1": "-12112785.5", "ECO2_2": "-5135821.052", "GUS": "6130182", "GUS1": "6712287", "GUS2": "6946755", "GCO2_1": "-699339", "GCO2_2": "-1566519.36", "WUS": "1849950.6", "WUS1": "2254096.6", "WUS2": "2179161.8", "WCO2_1": "-366678.6", "WCO2_2": "-121737.2952", "HUS": "26803.56", "HUS1": "21972.121", "HUS2": "24090.481", "HCO2_1": "3772.259", "HCO2_2": "6446.968818", "REG_DATE": "2021-03-11 18:31:32.0"}, {"YEAR": "2020", "MON": "12", "MM_TYPE": "개인", "CNT": "1110223", "EUS": "265370372", "EUS1": "250887378", "EUS2": "252983409", "ECO2_1": "13434978.5", "ECO2_2": "5696430.884", "GUS": "51095720", "GUS1": "50449214", "GUS2": "50298542", "GCO2_1": "721842", "GCO2_2": "1616926.08", "WUS": "15036546.3", "WUS1": "14479510.234", "WUS2": "14356727.09", "WCO2_1": "618427.638", "WCO2_2": "205317.975816", "HUS": "26512966.052", "HUS1": "23787438.214", "HUS2": "24469734.461", "HCO2_1": "2384379.7145", "HCO2_2": "4399125.3465152", "REG_DATE": "2021-03-11 18:31:32.0"}]}}'
data = json.loads(json_str)
rows = data.get('energyUseDataSummaryInfo', {}).get('row', [])
get_type_personal = lambda r : r['MM_TYPE'] == '개인'
data_personal_list = list(filter(get_type_personal, rows))
with open('test.json', mode='w') as f:
    f.write(f'{data_personal_list}')

In [ ]:
import json
columns = [
    'YEAR', #	년도
    'MON', #	월
    'MM_TYPE', #	회원타입
    'EUS', #	현년 전기사용량
    'GUS', #	현년 가스사용량
    'WUS', #	현년 수도사용량
    'HUS' #	현년 지역난방 사용량
]

data_personal_list = []
with open('data_personal_list.json', mode='r', encoding='utf-8') as f:
    filtered_list = json.load(f)
    # json_str = '{"energyUseDataSummaryInfo": {"list_total_count": 7, "RESULT": {"CODE": "INFO-000", "MESSAGE": "정상 처리되었습니다"}, "row": [{"YEAR": "2020", "MON": "12", "MM_TYPE": "공공기관", "CNT": "2319", "EUS": "134415822", "EUS1": "140830293", "EUS2": "143785616", "ECO2_1": "-7892132.5", "ECO2_2": "-3346264.18", "GUS": "3625280", "GUS1": "3938720", "GUS2": "4530840", "GCO2_1": "-609500", "GCO2_2": "-1365280", "WUS": "682469.6", "WUS1": "966115.2", "WUS2": "923345", "WCO2_1": "-262260.5", "WCO2_2": "-87070.486", "HUS": "0", "HUS1": "0", "HUS2": "0", "HCO2_1": "0", "HCO2_2": "0", "REG_DATE": "2021-03-11 18:31:32.0"}, {"YEAR": "2020", "MON": "12", "MM_TYPE": "기업", "CNT": "18259", "EUS": "567429666", "EUS1": "586794781", "EUS2": "593462122", "ECO2_1": "-22698785.5", "ECO2_2": "-9624285.052", "GUS": "24795758", "GUS1": "21567744", "GUS2": "26868220", "GCO2_1": "577776", "GCO2_2": "1294218.24", "WUS": "4056394.2", "WUS1": "4768906.8", "WUS2": "4628830", "WCO2_1": "-642474.2", "WCO2_2": "-213301.4344", "HUS": "0", "HUS1": "0", "HUS2": "0", "HCO2_1": "0", "HCO2_2": "0", "REG_DATE": "2021-03-11 18:31:32.0"}, {"YEAR": "2020", "MON": "12", "MM_TYPE": "소상공인", "CNT": "42743", "EUS": "82544954", "EUS1": "88007794", "EUS2": "87941049", "ECO2_1": "-5429467.5", "ECO2_2": "-2302094.22", "GUS": "2749449", "GUS1": "3195669", "GUS2": "3116983", "GCO2_1": "-406877", "GCO2_2": "-911404.48", "WUS": "1522971.8", "WUS1": "1710259.8", "WUS2": "1644036.2", "WCO2_1": "-154176.2", "WCO2_2": "-51186.4984", "HUS": "2679.61", "HUS1": "2096.78", "HUS2": "2244.49", "HCO2_1": "508.975", "HCO2_2": "-73.51022", "REG_DATE": "2021-03-11 18:31:32.0"}, {"YEAR": "2020", "MON": "12", "MM_TYPE": "학교", "CNT": "6457", "EUS": "148641247", "EUS1": "163043674", "EUS2": "158464391", "ECO2_1": "-12112785.5", "ECO2_2": "-5135821.052", "GUS": "6130182", "GUS1": "6712287", "GUS2": "6946755", "GCO2_1": "-699339", "GCO2_2": "-1566519.36", "WUS": "1849950.6", "WUS1": "2254096.6", "WUS2": "2179161.8", "WCO2_1": "-366678.6", "WCO2_2": "-121737.2952", "HUS": "26803.56", "HUS1": "21972.121", "HUS2": "24090.481", "HCO2_1": "3772.259", "HCO2_2": "6446.968818", "REG_DATE": "2021-03-11 18:31:32.0"}, {"YEAR": "2020", "MON": "12", "MM_TYPE": "개인", "CNT": "1110223", "EUS": "265370372", "EUS1": "250887378", "EUS2": "252983409", "ECO2_1": "13434978.5", "ECO2_2": "5696430.884", "GUS": "51095720", "GUS1": "50449214", "GUS2": "50298542", "GCO2_1": "721842", "GCO2_2": "1616926.08", "WUS": "15036546.3", "WUS1": "14479510.234", "WUS2": "14356727.09", "WCO2_1": "618427.638", "WCO2_2": "205317.975816", "HUS": "26512966.052", "HUS1": "23787438.214", "HUS2": "24469734.461", "HCO2_1": "2384379.7145", "HCO2_2": "4399125.3465152", "REG_DATE": "2021-03-11 18:31:32.0"}]}}'
    # data = json.loads(json_str)
    # rows = data.get('energyUseDataSummaryInfo', {}).get('row', [])
    # get_type_personal = lambda r : r['MM_TYPE'] == '개인'
    # filtered_list = list(filter(get_type_personal, rows))
    compact_list = [{col:row[col] for col in columns} for row in filtered_list]
    data_personal_list.extend(compact_list)
    print(data_personal_list)



with open('data_result.json', mode='w') as json_file:
    json.dump(data_personal_list, json_file, indent=4, ensure_ascii=False)

[{'YEAR': '2014', 'MON': '01', 'MM_TYPE': '개인', 'EUS': '144900337', 'GUS': '38335224', 'WUS': '10613959.838', 'HUS': '17632536.939'}, {'YEAR': '2014', 'MON': '03', 'MM_TYPE': '개인', 'EUS': '135419507', 'GUS': '32209021', 'WUS': '11069450.17', 'HUS': '10470340.034'}, {'YEAR': '2014', 'MON': '07', 'MM_TYPE': '개인', 'EUS': '168681507', 'GUS': '7318863', 'WUS': '13135941.064', 'HUS': '217853.383'}, {'YEAR': '2014', 'MON': '08', 'MM_TYPE': '개인', 'EUS': '187725750', 'GUS': '6596679', 'WUS': '14031651.912', 'HUS': '320553.853'}, {'YEAR': '2014', 'MON': '09', 'MM_TYPE': '개인', 'EUS': '175409826', 'GUS': '6969566', 'WUS': '13830683.654', 'HUS': '1041887.78'}, {'YEAR': '2014', 'MON': '11', 'MM_TYPE': '개인', 'EUS': '173263842', 'GUS': '19773109', 'WUS': '13489886.752', 'HUS': '10420879.095'}, {'YEAR': '2014', 'MON': '12', 'MM_TYPE': '개인', 'EUS': '180540444', 'GUS': '39598422', 'WUS': '13028824.456', 'HUS': '24632751.988'}, {'YEAR': '2015', 'MON': '01', 'MM_TYPE': '개인', 'EUS': '193784708', 'GUS': '591

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_json('data_result.json', orient='records', encoding='utf-8', lines=False)

df.describe(include='all')

df.describe(include='object')

df.describe()
df.describe(include=[np.number])

df['EUS'].describe()

,EUS
count,9.000000e+01
mean,2.468824e+08
std,5.909901e+07
min,1.348281e+08
25%,2.087638e+08
50%,2.445552e+08
75%,2.653565e+08
max,4.639713e+08
